Stratégie “gestion du risque” sur un panier de cryptos (BTC, ETH, BNB, SOL, XRP, DOGE). 

On part d’une allocation inverse-volatilité (moins on est volatile, plus on pèse), puis on ajuste l’exposition globale avec un vol targeting (objectif de volatilité annualisée). 

Ensuite, on ajoute deux filtres de régime : un mode stress (si la volatilité moyenne du marché est trop élevée, on coupe l’exposition) et un filtre de tendance BTC (si BTC est sous sa moyenne longue, on coupe aussi). 

Au final, on backteste cette allocation “risk-managed” et on la compare à une stratégie low-vol de base, avec des coûts de transaction via le turnover.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# params
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "DOGEUSDT"]
DATA_DIR = "binance_public_data"
START = "2018-01-01"
END   = "2025-12-31"

FREQ = 365
COST_BPS = 10

WIN_VOL = 20
WIN_PORTVOL = 60
MA_TREND = 200

VOL_TARGET = 0.60     # annual
MAX_LEV = 1.0         # cap
STRESS_Q = 0.90       # quantile stress

# load
def load_symbol(sym):
    fn = f"{DATA_DIR}/{sym}_1d_2021_2025.csv"  # adapte si besoin
    df = pd.read_csv(fn, parse_dates=["timestamp"], index_col="timestamp").sort_index()
    df = df.loc[START:END]
    df["ret"] = np.log(df["close"] / df["close"].shift(1))
    return df.dropna(subset=["ret"])

dfs = {s: load_symbol(s) for s in SYMBOLS}

# align
idx = None
for s in SYMBOLS:
    idx = dfs[s].index if idx is None else idx.intersection(dfs[s].index)

rets = pd.DataFrame({s: dfs[s].loc[idx, "ret"] for s in SYMBOLS}).dropna()
close = pd.DataFrame({s: dfs[s].loc[rets.index, "close"] for s in SYMBOLS})

# vols
vol = rets.rolling(WIN_VOL).std()

# inv-vol weights
w_inv = 1.0 / vol.replace(0, np.nan)
w_inv = w_inv.fillna(0.0)
w_inv = w_inv.div(w_inv.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# portfolio vol estimate
w_lag = w_inv.shift(1).fillna(0.0)
port_ret_gross = (w_lag * rets).sum(axis=1)
port_vol = port_ret_gross.rolling(WIN_PORTVOL).std() * np.sqrt(FREQ)

# vol targeting scalar
scale = (VOL_TARGET / port_vol).clip(lower=0.0, upper=MAX_LEV).fillna(0.0)

# stress regime
# stress = avg vol cross-section high
avg_vol = vol.mean(axis=1)
stress_th = avg_vol.quantile(STRESS_Q)
stress_off = (avg_vol > stress_th).astype(float)

# trend gate BTC
btc = close["BTCUSDT"]
btc_ma = btc.rolling(MA_TREND).mean()
trend_on = (btc > btc_ma).astype(float).fillna(0.0)

# final exposure gate
# 0 si stress ou trend off
gate = (1.0 - stress_off) * trend_on

# final weights
W = w_inv.mul(scale, axis=0).mul(gate, axis=0)

# normalize (after scaling)
W = W.clip(lower=0.0)
W = W.div(W.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# backtest
def run_bt(weights, rets, cost_bps=COST_BPS):
    w = weights.shift(1).reindex(rets.index).fillna(0.0)
    w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

    gross = (w * rets).sum(axis=1)

    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)

    net = gross - cost
    return net, to

def stats(r, freq=FREQ):
    r = r.dropna()
    eq = (1 + r).cumprod()
    mu = r.mean() * freq
    sig = r.std() * np.sqrt(freq)
    sharpe = mu / sig if sig > 0 else np.nan
    dd = (eq / eq.cummax() - 1.0).min()
    ann = eq.iloc[-1] ** (freq/len(r)) - 1 if len(r) > 0 else np.nan
    return float(ann), float(sig), float(sharpe), float(dd)

# baseline low_vol (rebuild quick)
q_cs = vol.quantile(0.5, axis=1)
w_low = (vol.le(q_cs, axis=0)).astype(float)
w_low = w_low.div(w_low.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

r_low, to_low = run_bt(w_low, rets, cost_bps=COST_BPS)
r_A, to_A     = run_bt(W, rets, cost_bps=COST_BPS)

ann, sig, sh, dd = stats(r_low)
print("baseline low_vol:", ann, sig, sh, dd, "turnover", float(to_low.mean()))
ann, sig, sh, dd = stats(r_A)
print("A invvol+target+stress+trend:", ann, sig, sh, dd, "turnover", float(to_A.mean()))

# plot
plt.figure(figsize=(12,5))
plt.plot((1+r_low.fillna(0)).cumprod(), label="low_vol")
plt.plot((1+r_A.fillna(0)).cumprod(), label="A allocation")
plt.legend()
plt.title("Equity")
plt.tight_layout()
plt.show()

print("gate on ratio:", float(gate.mean()))


baseline low_vol: 0.10257937374113202 0.6490640688547362 0.4818978015225074 -0.8168927380844624 turnover 0.0889497716894977

A invvol+target+stress+trend: 0.045805987841593954 0.41140330015429327 0.31616543892816273 -0.4583019050100299 turnover 0.04068328974335617

![Image test 9](image/image_test_9.png)


La stratégie “A” réduit fortement le risque : la volatilité annualisée baisse nettement (≈ 0.41 vs 0.65) et surtout le drawdown maximal est beaucoup moins sévère (≈ -46% vs -82%). En revanche, cette protection se paie en performance : le rendement annualisé devient plus faible (≈ 4.6% contre ≈ 10.3%) et le Sharpe baisse aussi. Cela suggère que les filtres (stress + tendance BTC) coupent une partie des phases haussières, ce qui réduit la participation au bull market mais améliore clairement la robustesse en période difficile.

Le gate on ratio ≈ 0.53 confirme que la stratégie est désinvestie environ la moitié du temps, ce qui explique la baisse de rendement mais aussi l’amélioration du drawdown. Enfin, le turnover moyen est plus faible que la baseline, donc la stratégie est aussi un peu plus “propre” en coûts.

gate on ratio: 0.5315068493150685